# 2. Loading the FE-ANN(s) EoS

The FE-ANN(s) EoS is implemented using `jax` and `flax`. The trained FE-ANN(s) EoS checkpoints are stored using `orbax`. First, we need to upload the required packages:
- **numpy**: numerical python
- **jax**: high-performance numerical computing and machine learning
- **nest_asyncio**: needed to load the checkpoints
- **feanns_eos**: library with implemented FE-ANN(s) EoS and phase equilibria solvers.

In [1]:
import numpy as np
from jax import numpy as jnp

from feanns_eos import HelmholtzModel
from feanns_eos import load_feanns_params
from feanns_eos import helper_get_alpha

from jax.config import config
config.update("jax_enable_x64", True)
type_np = np.float64
type_jax = jnp.float64

# needed to read checkpoints in jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

The checkpoints can be loaded with the `load_feanns_params` function. This function requires the folder (`ckpt_dir`) where the checkpoint is located and its name (`prefix`). The loaded state is a Python dictionary with some information about the FE-ANN(s) EoS, as shown below.

In [2]:
# directory where checkpoints are saved
ckpt_dir = "./eos_params"
# name of the checkpoint
prefix = "feanns_"

# loading the current checkpoint
state = load_feanns_params(ckpt_dir, prefix=prefix)
print('Information included in the checkpoint:', state.keys())

print("# epochs: ", state['epoch'])
print("features: ", state['features'], f'; this equals to {len(state["features"])} layers of {state["features"][0]} neurons')
print("seed: ", state['seed'])
print("learning_rate: ", state['learning_rate'])

Information included in the checkpoint: dict_keys(['epoch', 'features', 'learning_rate', 'params', 'seed'])
# epochs:  50000
features:  [40, 40, 40] ; this equals to 3 layers of 40 neurons
seed:  0
learning_rate:  1e-06


The `state['params']` value is a dictionary with the kernel and biases of the used in the FE-ANN(s) EoS

In [3]:
 state['params']

{'helmholtz_layer': {'kernel': array([[-1.91865365],
         [-3.42008273],
         [-2.40948245],
         [ 2.17382488],
         [ 1.87712497],
         [ 1.92623504],
         [ 1.98035204],
         [-1.99994356],
         [ 2.02550444],
         [ 0.75905708],
         [-1.8433767 ],
         [-2.47055048],
         [-0.52027889],
         [ 1.31926447],
         [ 1.3268983 ],
         [-1.33231753],
         [-2.69135919],
         [ 2.77863568],
         [ 0.42604366],
         [-2.42654057],
         [-8.27893431],
         [ 1.68126863],
         [-1.98010807],
         [ 0.83462859],
         [ 2.60469435],
         [-3.31947377],
         [-2.7173411 ],
         [ 1.19479068],
         [ 1.13801348],
         [ 2.43388489],
         [-1.27424681],
         [-1.80180761],
         [ 1.78896363],
         [-2.11985688],
         [-1.43679681],
         [ 1.33957413],
         [-1.06817372],
         [ 2.47614909],
         [ 0.91262327],
         [-0.47107328]])},
 'hidden

To create an instance of the FE-ANN(s) EoS, you need the list of features (`state['features']`). The parameters (`state['params']`) will be needed to use the model.

In [4]:
# creating the model with the given features
model = HelmholtzModel(features=state['features'])
params = {'params': state['params']}

The `model.apply` function evaluates the model; this function requires the parameters and the model inputs (alpha, rhoad, and Tad). The alpha parameter is used as the descriptor of the Mie potential and can be computed with the `helper_get_alpha` function.

$$ \alpha_{\mathrm{vdw}} = \mathcal{C}^{\mathrm{Mie}} \left( \frac{1}{\lambda_a - 3} - \frac{1}{\lambda_r - 3}\right) $$

By default, the method `model.apply` computes the residual Helmholtz free energy. See the next notebook for examples of how to get other thermophysical properties.

In [5]:
lr = np.array([12])
la = np.array([6])
alpha = helper_get_alpha(lr, la)
rhoad = np.array([0.8])
Tad = np.array([2.4])

# computing the residual Helmholtz free energy
model.apply(params, alpha, rhoad, Tad)

Array([1.12845457], dtype=float64)

The FE-ANN(s) EoS is already parallelized. Multiple state points can be evaluated per call. The inputs must have the same length.

In [6]:
lr = np.array([12, 12])
la = np.array([6, 6])
alpha = helper_get_alpha(lr, la)
rhoad = np.array([0.8, 0.135])
Tad = np.array([2.4, 1.4])

# computing the residual Helmholtz free energy
model.apply(params, alpha, rhoad, Tad)

Array([ 1.12845457, -0.50372673], dtype=float64)